# Taxi Dataset

Download the CSV Files from: 
* 2018 yellow cab taxi rides (112M lines; may sample down to 1M for convenience): https://data.cityofnewyork.us/Traasportation/2018-Yellow-Taxi-Trip-Data/t29m-gskq
* taxi zone names and polygons: https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc

Store them in this directory ("datasets") as `taxis.csv` and `taxi_zones.csv`, respectively.

### Transform/Filter the data for use with the sampling pipeline

#### Precompute polygons

In [ ]:
import pandas as pd
import numpy as np
import random
from shapely.wkt import loads
from shapely.geometry import MultiPolygon, Point

rides_df = pd.read_csv("./taxis.csv", delimiter=";", parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"])
zones_df = pd.read_csv("./taxi_zones.csv")


zones_df = zones_df[["LocationID", "the_geom"]].set_index("LocationID")
rides_df = rides_df.join(zones_df, on="PULocationID")
rides_df = rides_df.join(zones_df, on="DOLocationID", rsuffix="DO")


# only keep rows with a dropoff and a pickup point (i.e., no NaN entries)
rides_df = rides_df[rides_df["the_geom"].notnull() & rides_df["the_geomDO"].notnull()]

# only keep rows with a pickup and dropoff date in 2018 as simple data cleaning (but there are some 
# rides around New Year that get discarded, though)
rides_df = rides_df[rides_df["tpep_pickup_datetime"].dt.year == 2018]
rides_df = rides_df[rides_df["tpep_dropoff_datetime"].dt.year == 2018]

# make temporal dimensions numeric (some parts of the pipeline break when data is string or object)
rides_df["tpep_pickup_datetime"] = rides_df["tpep_pickup_datetime"].astype(np.int64)
rides_df["tpep_dropoff_datetime"] = rides_df["tpep_dropoff_datetime"].astype(np.int64)

def generate_representative(poly: MultiPolygon):
  '''Computes a random location inside the polygon until that location actually lies inside the 
     polygon. This random location is used as a representative point for the polygon in the 
     dataset.'''
  min_x, min_y, max_x, max_y = poly.bounds
  pos = [-1, -1]
  is_inside = False
  while not is_inside:
    pos[0] = random.uniform(min_x, max_x)
    pos[1] = random.uniform(min_y, max_y)
    is_inside = poly.contains(Point(pos))
  return np.array(pos).reshape(2, 1)

# generate a representative geo point for each taxi ride's pick-up and drop-off location, so that 
# we can linearize by geospace
PUrepresentatives = rides_df.apply(
  lambda row: generate_representative(loads(row["the_geom"])), 
  axis=1
)

# np.stack turns array of objects into array of arrays
PUrepresentatives = np.stack(PUrepresentatives)

DOrepresentatives = rides_df.apply(
  lambda row: generate_representative(loads(row["the_geomDO"])), 
  axis=1
)
DOrepresentatives = np.stack(DOrepresentatives)

# store lat and lng as separate attributes
rides_df["PUrepresentativeX"] = PUrepresentatives[:, 0]
rides_df["PUrepresentativeY"] = PUrepresentatives[:, 1]
rides_df["DOrepresentativeX"] = DOrepresentatives[:, 0]
rides_df["DOrepresentativeY"] = DOrepresentatives[:, 1]

# drop unneeded geometry and non-numerical cols
rides_df.drop(columns=["the_geom", "the_geomDO", "store_and_fwd_flag"], inplace=True)

# write the update data back to the file
rides_df.to_csv("./taxisData.csv", sep=";", index=False, header=False)
# takes about 15:30 minutes for 1M 

#### Precompute spatial autocorrelation (run the previous cell first!)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree

# reproduces the autocorrelation computation outlined here: 
# https://geographicdata.science/book/notebooks/06_spatial_autocorrelation.html

# find the knn for each point
columns = ["tripID", "VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count", "trip_distance", "RatecodeID", "PULocationID", "DOLocationID", "payment_type", "fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge", "total_amount", "PURepresentativeX", "PURepresentativeY", "DORepresentativeX", "DORepresentativeY"]

rides_df = pd.read_csv("./taxisData.csv", delimiter=";", names=columns)


X = rides_df.to_numpy()

# constants for indeces of important columns in the np array 
LAT = 17
LON = 18
TRIP_DISTANCE = 5
TRIP_ID = 0

tree = KDTree(X[:, [LAT, LON]])
dist, knn = tree.query(X[:, [LAT, LON]], k=9)  # using k=8 (+1, as first match is item itself)

# first match to query is the item itself, so remove that one
dist = dist[:, 1:]
knn = knn[:, 1:]

# compute the weights for the knn per point (the closer, the greater the weight)
w = 1 / dist 
w = w / w.sum(axis=1).reshape(-1, 1)

# compute the weighted knn value per point (i.e., the spatial lag)
knn_value = X[knn][:, :, TRIP_DISTANCE]  # get the value column per neighbor
spatial_lag = knn_value * w  # multiply the value vector with weights
spatial_lag = spatial_lag.sum(axis=1)

# compute the normalized actual value
mean_value = np.mean(X[:, TRIP_DISTANCE], axis=0)
normalized_value = X[:, TRIP_DISTANCE] - mean_value

# compute the normalized lag value
mean_spatial_lag = np.mean(spatial_lag, axis=0)
normalized_spatial_lag = spatial_lag - mean_spatial_lag

# compute the H/L labels, indicating the quadrants in the Moran's plot
value_is_H = normalized_value > 0
spatial_lag_is_H = normalized_spatial_lag > 0
# value_is_H.astype(int) + spatial_lag_is_H.astype(int)

spatial_auto_df = pd.DataFrame(
  np.array([
    X[:, TRIP_ID], 
    normalized_value,
    normalized_spatial_lag,
    value_is_H,
    spatial_lag_is_H,
  ]).T,
  columns=[
    "tripID",
    "normalized_value",
    "normalized_spatial_lag",
    "value_is_H",
    "spatial_lag_is_H"
  ],
).set_index("tripID")
rides_df.astype({"tripID": int}).set_index("tripID").join(spatial_auto_df).to_csv("taxisData.csv", sep=";", index=True, header=False)

Moran Plot:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# https://geographicdata.science/book/notebooks/06_spatial_autocorrelation.html
fig, ax = plt.subplots(1, figsize=(12, 6))
sns.regplot(
  x=normalized_value,
  y=normalized_spatial_lag,
  # ci=None,
  color="r",
  scatter_kws={"s": 0.1, "alpha": 0.3}
)
ax.axvline(0, c="k", alpha=0.5)
ax.axhline(0, c="k", alpha=0.5)
ax.set_title('Moran Plot - trip_distance')
plt.show()

### Dataset summary

In [ ]:
import pandas as pd

columns = ["tripID", "VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count", "trip_distance", "RatecodeID", "PULocationID", "DOLocationID", "payment_type", "fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge", "total_amount", "PURepresentativeX", "PURepresentativeY", "DORepresentativeX", "DORepresentativeY", "normalized_value", "normalized_spatial_lag", "value_is_H", "spatial_lag_is_H"]

rides_df = pd.read_csv("./taxisData.csv", sep=";", names=columns)
pd.set_option('display.max_columns', 500)
rides_df.describe().loc[["min", "max"]]

## Precompute linearizations
WE precompute linearizations and store them in CSV files in `../linearization_files/` to reduce loading times.

***Note** that you must run this cell to recreate the figures in `figures.ipynb`.*

In [ ]:
import pandas as pd
import sys
import os

sys.path.append(f"{os.getcwd()}/../../../")  # HACK to make relative imports work in notebooks

from pipeline.linearizations.Linearization import LinearizationDatetimeAttr, LinearizationNumericAttr, LinearizationGeoZorder, LinearizationRandom

LinearizationRandom("taxis", 25).linearize()  # random shuffling
LinearizationDatetimeAttr("taxis", 25, 2).linearize()  # sort by pickup date
LinearizationNumericAttr("taxis", 25, 5).linearize()  # sort by trip distance
LinearizationGeoZorder("taxis", 25, 17, 18).linearize()  # sort by pickup location

"done"

### Plot distribution of pickup location

In [ ]:
# plot the rides dataset in hexbinned histogram
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111)

header = ["tripID", "VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count", "trip_distance", "RatecodeID", "PULocationID", "DOLocationID", "payment_type", "fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge", "total_amount", "PURepresentativeX", "PURepresentativeY", "DORepresentativeX", "DORepresentativeY", "normalized_value", "normalized_spatial_lag", "value_is_H", "spatial_lag_is_H"]

df = pd.read_csv("./taxisData.csv", delimiter=";", names=header)

image = plt.hexbin(
  x=df["PURepresentativeX"], 
  y=df["PURepresentativeY"],
  bins="log",
  mincnt=1,
  gridsize=100,
)

plt.grid(True, )
# to get the computed bins, use 
# image.get_array()

### Plot the time distribution

In [ ]:
# plot the rides dataset in hexbinned histogram
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(111)

header = ["tripID", "VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count", "trip_distance", "RatecodeID", "PULocationID", "DOLocationID", "payment_type", "fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge", "total_amount", "PURepresentativeX", "PURepresentativeY", "DORepresentativeX", "DORepresentativeY", "normalized_value", "normalized_spatial_lag", "value_is_H", "spatial_lag_is_H"]

df = pd.read_csv("./taxisData.csv", delimiter=";", names=header)

plt.hist(
  pd.to_datetime(df["tpep_pickup_datetime"]), 
  bins=12,
  range=('2018-01-01 00:00:00', '2018-12-31 23:59:59'),
  edgecolor="black",
  linewidth=1
)
"done"

### Plot the trip distance distribution

In [ ]:
# plot the rides dataset in hexbinned histogram
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(111)

header = ["tripID", "VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count", "trip_distance", "RatecodeID", "PULocationID", "DOLocationID", "payment_type", "fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge", "total_amount", "PURepresentativeX", "PURepresentativeY", "DORepresentativeX", "DORepresentativeY", "normalized_value", "normalized_spatial_lag", "value_is_H", "spatial_lag_is_H"]

df = pd.read_csv("./taxisData.csv", delimiter=";", names=header)

plt.hist(
  df["trip_distance"], 
  bins=100,
  range=(0, 30),
  edgecolor="black",
  linewidth=1
)
"done"

### Plot a linearization:

In [ ]:
import pandas as pd
import numpy as np

header = ["tripID", "VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count", "trip_distance", "RatecodeID", "PULocationID", "DOLocationID", "payment_type", "fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge", "total_amount", "PURepresentativeX", "PURepresentativeY", "DORepresentativeX", "DORepresentativeY", "normalized_value", "normalized_spatial_lag", "value_is_H", "spatial_lag_is_H"]

df = pd.read_csv("../../linearization_files/taxisLinearizationZOrder.csv", delimiter=";", names=header)
sample = np.unique(np.random.randint(low=0, high=len(df) - 1, size=100000))
sample = sample[np.argsort(sample)]
df.iloc[sample].plot.line(x="PURepresentativeX", y="PURepresentativeY", figsize=[15, 15], c="black", lw=0.15)